In [1]:
# Imports

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
import calendar
from datetime import datetime
from jours_feries_france import JoursFeries

In [2]:
# La fonction renvoie sous forme de dataframe, le taux d'occupation et le débit pour chaque heure de la journée


def get_school_holidays(year):

    # API Url
    api_url = "https://data.education.gouv.fr/api/explore/v2.1/catalog/datasets/fr-en-calendrier-scolaire/records"

    # Request parameters
    params = {
        "limit": -1,
        #"order_by": "t_1h",
        "timezone": "CET",
        "where": f'location="Paris" AND (annee_scolaire = "{str(year-1)}-{str(year)}" OR annee_scolaire = "{str(year)}-{str(year+1)}")'
    }

    # Get response
    response = requests.get(api_url, params=params)

    # Fetch data
    if response.status_code == 200:
        records = response.json().get("results", [])
        if records:
            start = []
            end = []
            for index, record in enumerate(records):
                start_date = record["start_date"]
                syear = int(start_date[:4])
                smonth = int(start_date[5:7])
                sday = int(start_date[8:10])
                end_date = record["end_date"]
                eyear = int(end_date[:4])
                emonth = int(end_date[5:7])
                eday = int(end_date[8:10])
                
                start.append(datetime(syear, smonth, sday))
                end.append(datetime(eyear, emonth, eday))
        else:
            print("No data found.")
    else:
        print(f"API Error : {response.status_code}")

    return start, end

def get_public_holidays(year):
    dates = JoursFeries.for_year(year)
    return list(dates.values())


print(get_school_holidays(2024))
print(get_public_holidays(2024))

([datetime.datetime(2024, 12, 21, 0, 0), datetime.datetime(2025, 4, 12, 0, 0), datetime.datetime(2024, 5, 9, 0, 0), datetime.datetime(2025, 5, 29, 0, 0), datetime.datetime(2023, 12, 23, 0, 0), datetime.datetime(2024, 2, 10, 0, 0), datetime.datetime(2025, 7, 5, 0, 0), datetime.datetime(2024, 4, 6, 0, 0), datetime.datetime(2023, 10, 21, 0, 0), datetime.datetime(2024, 7, 6, 0, 0), datetime.datetime(2024, 7, 6, 0, 0), datetime.datetime(2024, 10, 19, 0, 0), datetime.datetime(2025, 2, 15, 0, 0), datetime.datetime(2025, 7, 5, 0, 0)], [datetime.datetime(2025, 1, 6, 0, 0), datetime.datetime(2025, 4, 28, 0, 0), datetime.datetime(2024, 5, 13, 0, 0), datetime.datetime(2025, 6, 2, 0, 0), datetime.datetime(2024, 1, 8, 0, 0), datetime.datetime(2024, 2, 26, 0, 0), datetime.datetime(2025, 8, 29, 0, 0), datetime.datetime(2024, 4, 22, 0, 0), datetime.datetime(2023, 11, 6, 0, 0), datetime.datetime(2024, 9, 2, 0, 0), datetime.datetime(2024, 8, 30, 0, 0), datetime.datetime(2024, 11, 4, 0, 0), datetime.datet

In [3]:
def get_holidays_df(target_year):
    dates = pd.date_range(start=f"{target_year}-01-01", end=f"{target_year}-12-31", freq="D")

    df = pd.DataFrame({
        "year": dates.year,
        "month": dates.month,
        "day": dates.day,
        "school_holiday": [0] * len(dates),  # Première colonne vide
        "public_holiday": [0] * len(dates)   # Deuxième colonne vide
    })

    start_dates, end_dates = get_school_holidays(target_year)

    for index in range(len(start_dates)):
        sday = start_dates[index]
        eday = end_dates[index]

        sh_days = pd.date_range(start=sday, end=eday, freq="D")
        sh_days_list = sh_days.tolist()

        for sh_day in sh_days_list:
            year, month, day = sh_day.year, sh_day.month, sh_day.day
            if year != target_year:
                continue
            else:
                mask = (df["year"] == year) & (df["month"] == month) & (df["day"] == day)
                df.loc[mask, "school_holiday"] = 1
        
        ph_days = get_public_holidays(target_year)
        for ph_day in ph_days:
            year, month, day = ph_day.year, ph_day.month, ph_day.day
            mask = (df["year"] == year) & (df["month"] == month) & (df["day"] == day)
            df.loc[mask, "public_holiday"] = 1

    return df

holidays_df = get_holidays_df(2024)
print(holidays_df[:30])
print(holidays_df[-30:])

    year  month  day  school_holiday  public_holiday
0   2024      1    1               1               1
1   2024      1    2               1               0
2   2024      1    3               1               0
3   2024      1    4               1               0
4   2024      1    5               1               0
5   2024      1    6               1               0
6   2024      1    7               1               0
7   2024      1    8               1               0
8   2024      1    9               0               0
9   2024      1   10               0               0
10  2024      1   11               0               0
11  2024      1   12               0               0
12  2024      1   13               0               0
13  2024      1   14               0               0
14  2024      1   15               0               0
15  2024      1   16               0               0
16  2024      1   17               0               0
17  2024      1   18               0          